# 1. Import

In [ ]:
import pandas as pd
from pathlib import Path
import openpyxl
import string

import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import sparse
import joblib

DATA_PATH = Path("../data/")

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# 2. Load Data

In [3]:
headlines_df = pd.read_csv(DATA_PATH / "NVIDIA_NewsHeadlines_20241101-Present.csv")

stock_df = pd.read_excel(DATA_PATH / "NVIDIA_StockPriceDaily_20241101-Present.xlsx", engine='openpyxl', header=None)

# Now manually assign column names:
stock_df.columns = [
    'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume'
]

stock_df["date"] = pd.to_datetime(stock_df["date"]).dt.date
headlines_df["date"] = pd.to_datetime(headlines_df["seendate"]).dt.date

In [4]:
stock_df.head()

,date,open,high,low,close,adj_close,volume
0,2025-11-21,181.24,184.56,172.93,178.88,178.88,346068500
1,2025-11-20,195.95,196.00,179.85,180.64,180.64,343504800
2,2025-11-19,184.79,187.86,182.83,186.52,186.52,247246400
3,2025-11-18,183.38,184.80,179.65,181.36,181.36,213598900
4,2025-11-17,185.97,189.00,184.32,186.60,186.60,172821100


In [5]:
headlines_df.head()

,title,seendate,url,sourcecountry,language,domain,date
0,Stock market suffers a Halloween selloff as te...,2024-11-01 01:30:00+00:00,https://www.morningstar.com/news/marketwatch/2...,China,English,morningstar.com,2024-11-01
1,Why Nvidia stock is sinking today,2024-11-01 01:30:00+00:00,https://www.fool.com.au/2024/11/01/why-nvidia-...,Australia,English,fool.com.au,2024-11-01
2,US close : Stocks sharply lower following tech...,2024-11-01 01:45:00+00:00,https://www.sharecast.com/news/market-report-u...,United Kingdom,English,sharecast.com,2024-11-01
3,Why Nvidia Stock Is Sinking Today | The Motley...,2024-11-01 02:15:00+00:00,https://www.fool.com/investing/2024/10/31/why-...,United States,English,fool.com,2024-11-01
4,Billionaire Philippe Laffont of Coatue Is Dump...,2024-11-01 03:00:00+00:00,https://finance.yahoo.com/news/billionaire-phi...,United States,English,finance.yahoo.com,2024-11-01


In [6]:
merged_df = pd.merge(headlines_df, stock_df, on='date', how='inner')
desired_order = [
    'language',
    'sourcecountry',
    'seendate',
    'date',
    'url',
    'title',
    'domain',
    'open',
    'high',
    'low',
    'close',
    'adj_close',
    'volume'
]

merged_df = merged_df[desired_order]
merged_df['date'] = pd.to_datetime(merged_df['date'])

merged_df.head()

,language,sourcecountry,seendate,date,url,title,domain,open,high,low,close,adj_close,volume
0,English,Australia,2024-11-18 03:45:00+00:00,2024-11-18,https://www.fool.com.au/2024/11/18/prediction-...,Prediction : Nvidia stock is going to soar aft...,fool.com.au,139.5,141.55,137.15,140.15,140.11,221205300
1,English,Cyprus,2024-11-18 04:00:00+00:00,2024-11-18,https://cyprus-mail.com/2024/11/18/softbank-fi...,SoftBank first to receive new Nvidia chips for...,cyprus-mail.com,139.5,141.55,137.15,140.15,140.11,221205300
2,English,China,2024-11-18 04:00:00+00:00,2024-11-18,https://www.morningstar.com/markets/this-unlov...,Why Small - Cap Value Stocks Look Attractive R...,morningstar.com,139.5,141.55,137.15,140.15,140.11,221205300
3,English,United States,2024-11-18 06:30:00+00:00,2024-11-18,https://247wallst.com/market-news/2024/11/17/n...,Nasdaq Futures Up Sunday Night : NVIDIA Earnin...,247wallst.com,139.5,141.55,137.15,140.15,140.11,221205300
4,English,United States,2024-11-18 11:00:00+00:00,2024-11-18,https://www.benzinga.com/24/11/42029943/dow-tu...,Dow Tumbles Over 300 Points Following Economic...,benzinga.com,139.5,141.55,137.15,140.15,140.11,221205300


In [7]:
merged_df.to_csv(DATA_PATH / "NVIDIA_Merged_20241101-Present.csv", index=False)

# 3. Split Data

In [8]:
# -------------------------------------------------------------------
# Train / test split by date (training = pre 2025-11-01)
# -------------------------------------------------------------------
SPLIT_DATE = pd.to_datetime('2025-11-01')

train_df = merged_df[merged_df['date'] < SPLIT_DATE].copy()
test_df  = merged_df[merged_df['date'] >= SPLIT_DATE].copy()

# 4. Text Cleaning for TF IDF

In [9]:
# -------------------------------------------------------------------
# 2. Text cleaning
# -------------------------------------------------------------------

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# Add nvidia variants to custom stopwords
custom_stop = {"nvidia", "nvidias", "nvdia", "NVDA", "nvda", "nasdaq"}

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)

    tokens = nltk.word_tokenize(text)

    tokens = [w for w in tokens if w not in stop_words and w not in custom_stop]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return ' '.join(tokens)

train_df["cleaned_title"] = [preprocess_text(t) for t in train_df["title"]]
test_df["cleaned_title"]  = [preprocess_text(t) for t in test_df["title"]]

In [10]:
train_daily = (
    train_df
    .groupby('date')['cleaned_title']
    .apply(lambda x: " ".join(x))
    .reset_index()
    .sort_values('date')
)

test_daily = (
    test_df
    .groupby('date')['cleaned_title']
    .apply(lambda x: " ".join(x))
    .reset_index()
    .sort_values('date')
)

# 5. TFIDF on Training data and Save

In [11]:
# -------------------------------------------------------------------
# 2) TF-IDF on TRAIN, then transform TEST with same vectorizer
# -------------------------------------------------------------------
vectorizer = TfidfVectorizer(
    max_features=50,      # you can tune this
    ngram_range=(1, 2),     # unigrams + bigrams
    min_df=2                # ignore super-rare tokens
)

X_train_text = vectorizer.fit_transform(train_daily['cleaned_title'])
X_test_text  = vectorizer.transform(test_daily['cleaned_title'])

In [14]:
# Save sparse TF-IDF matrices
sparse.save_npz(f"{DATA_PATH}/tfidf/X_train_tfidf.npz", X_train_text)
sparse.save_npz(f"{DATA_PATH}/tfidf/X_test_tfidf.npz", X_test_text)

# Save the vectorizer (vocabulary + settings)
joblib.dump(vectorizer, f"{DATA_PATH}/tfidf/tfidf_vectorizer.pkl")

['..\\data/tfidf/tfidf_vectorizer.pkl']